<a href="https://colab.research.google.com/github/VriskaSerket51/idbot/blob/main/Tchatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 트랜스포머 구현

In [5]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스(2i)에는 사인 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])

    # 배열의 홀수 인덱스(2i+1)에는 코사인 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:, 0::2] = sines
    angle_rads[:, 1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]

In [6]:
def scaled_dot_product_attention(query, key, value, mask):
  # query 크기 : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  # key 크기 : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
  # value 크기 : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
  # padding_mask : (batch_size, 1, 1, key의 문장 길이)

  # Q와 K의 곱. 어텐션 스코어 행렬.
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 스케일링
  # dk의 루트값으로 나눠준다.
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 마스킹. 어텐션 스코어 행렬의 마스킹 할 위치에 매우 작은 음수값을 넣는다.
  # 매우 작은 값이므로 소프트맥스 함수를 지나면 행렬의 해당 위치의 값은 0이 된다.
  if mask is not None:
    logits += (mask * -1e9)

  # 소프트맥스 함수는 마지막 차원인 key의 문장 길이 방향으로 수행된다.
  # attention weight : (batch_size, num_heads, query의 문장 길이, key의 문장 길이)
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # output : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
  output = tf.matmul(attention_weights, value)

  return output, attention_weights

In [7]:
class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    # d_model을 num_heads로 나눈 값.
    # 논문 기준 : 64
    self.depth = d_model // self.num_heads

    # WQ, WK, WV에 해당하는 밀집층 정의
    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    # WO에 해당하는 밀집층 정의
    self.dense = tf.keras.layers.Dense(units=d_model)

  # num_heads 개수만큼 q, k, v를 split하는 함수
  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # 1. WQ, WK, WV에 해당하는 밀집층 지나기
    # q : (batch_size, query의 문장 길이, d_model)
    # k : (batch_size, key의 문장 길이, d_model)
    # v : (batch_size, value의 문장 길이, d_model)
    # 참고) 인코더(k, v)-디코더(q) 어텐션에서는 query 길이와 key, value의 길이는 다를 수 있다.
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 2. 헤드 나누기
    # q : (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    # k : (batch_size, num_heads, key의 문장 길이, d_model/num_heads)
    # v : (batch_size, num_heads, value의 문장 길이, d_model/num_heads)
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 3. 스케일드 닷 프로덕트 어텐션. 앞서 구현한 함수 사용.
    # (batch_size, num_heads, query의 문장 길이, d_model/num_heads)
    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    # (batch_size, query의 문장 길이, num_heads, d_model/num_heads)
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 4. 헤드 연결(concatenate)하기
    # (batch_size, query의 문장 길이, d_model)
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 5. WO에 해당하는 밀집층 지나기
    # (batch_size, query의 문장 길이, d_model)
    outputs = self.dense(concat_attention)

    return outputs

In [8]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 멀티-헤드 어텐션 (첫번째 서브층 / 셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': padding_mask # 패딩 마스크 사용
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 포지션 와이즈 피드 포워드 신경망 (두번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [9]:
def encoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 인코더는 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 인코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [10]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")

  # 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")

  # 패딩 마스크(두번째 서브층)
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 멀티-헤드 어텐션 (첫번째 서브층 / 마스크드 셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs, 'key': inputs, 'value': inputs, # Q = K = V
          'mask': look_ahead_mask # 룩어헤드 마스크
      })

  # 잔차 연결과 층 정규화
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 멀티-헤드 어텐션 (두번째 서브층 / 디코더-인코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1, 'key': enc_outputs, 'value': enc_outputs, # Q != K = V
          'mask': padding_mask # 패딩 마스크
      })

  # 드롭아웃 + 잔차 연결과 층 정규화
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 포지션 와이즈 피드 포워드 신경망 (세번째 서브층)
  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 드롭아웃 + 잔차 연결과 층 정규화
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [11]:
def decoder(vocab_size, num_layers, dff,
            d_model, num_heads, dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')

  # 디코더는 룩어헤드 마스크(첫번째 서브층)와 패딩 마스크(두번째 서브층) 둘 다 사용.
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 포지셔널 인코딩 + 드롭아웃
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # 디코더를 num_layers개 쌓기
  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads,
        dropout=dropout, name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [12]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, key의 문장 길이)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [13]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x) # 패딩 마스크도 포함
  return tf.maximum(look_ahead_mask, padding_mask)

In [14]:
def transformer(vocab_size, num_layers, dff,
                d_model, num_heads, dropout,
                name="transformer"):

  # 인코더의 입력
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 디코더의 입력
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더의 패딩 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더의 룩어헤드 마스크(첫번째 서브층)
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask, output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 디코더의 패딩 마스크(두번째 서브층)
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더의 출력은 enc_outputs. 디코더로 전달된다.
  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[inputs, enc_padding_mask]) # 인코더의 입력은 입력 문장과 패딩 마스크

  # 디코더의 출력은 dec_outputs. 출력층으로 전달된다.
  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff,
      d_model=d_model, num_heads=num_heads, dropout=dropout,
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 다음 단어 예측을 위한 출력층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [15]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [16]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

# 챗봇 구현

In [19]:
!git clone https://github.com/VriskaSerket51/idbot

fatal: destination path 'idbot' already exists and is not an empty directory.


In [20]:
data = pd.read_csv('idbot/ChatBotData.csv')
data.tail()

,Q,A,Rule?
9029,이번 주 행사 뭐 있어?,<Rule>,14
9030,행사가… 뭐가 있더라?,<Rule>,14
9031,이번주 무슨 행사 있다고?,<Rule>,14
9032,행사가?,<Rule>,14
9033,행사… 행사 좀…,<Rule>,14


In [21]:
train_data=data[data['Rule?']==0]
train_data.tail()

,Q,A,Rule?
8862,파이를 끝까지 써봐.,3.141592653589793238462643... 더 길다란 건 구글에게 물...,0
8863,몰?,루,0
8864,너는 몇살이니?,저는 한 살 이에요.,0
8865,23기 최고 잘생긴 사람은?,김준수,0
8866,안녕?,안녕하세요. 저는 이디저디 전담 챗봇 이디봇이라고 해요.,0


In [22]:
print('챗봇 샘플의 개수 :', len(train_data))

챗봇 샘플의 개수 : 8867


In [23]:
questions = []
for sentence in train_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [24]:
answers = []
for sentence in train_data['A']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [25]:
print(questions[:5])
print(answers[:5])

['12시 땡 !', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
['하루가 또 가네요 .', '위로해 드립니다 .', '여행은 언제나 좋죠 .', '여행은 언제나 좋죠 .', '눈살이 찌푸려지죠 .']


In [26]:
Q=data["Q"]
A=data["A"]
s=Q
sentences=[]
for i in Q:
  sentences.append(i)
for i in A:
  if i!="<Rule>":
    sentences.append(i)

In [27]:
print(sentences[-6:])

[' anyo', '  3.141592653589793238462643... 더 길다란 건 구글에게 물어보세요.', ' 루', ' 저는 한 살 이에요.', '김준수', ' 안녕하세요. 저는 이디저디 전담 챗봇 이디봇이라고 해요.']


In [28]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    sentences, target_vocab_size=2**13)

In [29]:
# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2

In [30]:
print('시작 토큰 번호 :',START_TOKEN)
print('종료 토큰 번호 :',END_TOKEN)
print('단어 집합의 크기 :',VOCAB_SIZE)

시작 토큰 번호 : [8133]
종료 토큰 번호 : [8134]
단어 집합의 크기 : 8135


In [31]:
# 서브워드텍스트인코더 토크나이저의 .encode()를 사용하여 텍스트 시퀀스를 정수 시퀀스로 변환.
print('임의의 질문 샘플을 정수 인코딩 : {}'.format(tokenizer.encode(questions[20])))

임의의 질문 샘플을 정수 인코딩 : [7832, 4853, 271, 421, 5476, 879]


In [32]:
# 서브워드텍스트인코더 토크나이저의 .encode()와 .decode() 테스트해보기
# 임의의 입력 문장을 sample_string에 저장
sample_string = questions[20]

# encode() : 텍스트 시퀀스 --> 정수 시퀀스
tokenized_string = tokenizer.encode(sample_string)
print ('정수 인코딩 후의 문장 {}'.format(tokenized_string))

# decode() : 정수 시퀀스 --> 텍스트 시퀀스
original_string = tokenizer.decode(tokenized_string)
print ('기존 문장: {}'.format(original_string))

정수 인코딩 후의 문장 [7832, 4853, 271, 421, 5476, 879]
기존 문장: 가스비 비싼데 감기 걸리겠어


In [33]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 40

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

In [34]:
questions, answers = tokenize_and_filter(questions, answers)

In [35]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [36]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1

model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

(1, 8135, 256)
(1, 8135, 256)


In [37]:
model.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 enc_padding_mask (Lambda)      (None, 1, 1, None)   0           ['inputs[0][0]']                 
                                                                                                  
 encoder (Functional)           (None, None, 256)    3136768     ['inputs[0][0]',                 
                                                                  'enc_padding_mask[0][0

In [38]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [39]:
EPOCHS = 20
model.fit(dataset, epochs=EPOCHS)

Epoch 1/20
139/139 [==============================] - 17s 57ms/step - loss: 1.4249 - accuracy: 0.0193
Epoch 2/20
139/139 [==============================] - 8s 57ms/step - loss: 1.2242 - accuracy: 0.0447
Epoch 3/20
139/139 [==============================] - 8s 56ms/step - loss: 1.0311 - accuracy: 0.0496
Epoch 4/20
139/139 [==============================] - 8s 56ms/step - loss: 0.9237 - accuracy: 0.0516
Epoch 5/20
139/139 [==============================] - 8s 56ms/step - loss: 0.8594 - accuracy: 0.0556
Epoch 6/20
139/139 [==============================] - 8s 59ms/step - loss: 0.8051 - accuracy: 0.0584
Epoch 7/20
139/139 [==============================] - 8s 57ms/step - loss: 0.7515 - accuracy: 0.0627
Epoch 8/20
139/139 [==============================] - 8s 57ms/step - loss: 0.6949 - accuracy: 0.0685
Epoch 9/20
139/139 [==============================] - 8s 58ms/step - loss: 0.6326 - accuracy: 0.0757
Epoch 10/20
139/139 [==============================] - 8s 58ms/step - loss: 0.5651 - accur

In [40]:
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [41]:
def evaluate(sentence):
  # 입력 문장에 대한 전처리
  sentence = preprocess_sentence(sentence)

  # 입력 문장에 시작 토큰과 종료 토큰을 추가
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  for i in range(MAX_LENGTH):
    predictions = model(inputs=[sentence, output], training=False)

    # 현재 시점의 예측 단어를 받아온다.
    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    # 현재 시점의 예측 단어를 output(출력)에 연결한다.
    # output은 for문의 다음 루프에서 디코더의 입력이 된다.
    output = tf.concat([output, predicted_id], axis=-1)

  # 단어 예측이 모두 끝났다면 output을 리턴.
  return tf.squeeze(output, axis=0)

In [42]:
def predict(sentence):
  prediction = evaluate(sentence)

  # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  # print('Input: {}'.format(sentence))
  # print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [43]:
output = predict("내일 일본어 수행 있어.")

In [44]:
model.save_weights('transformer')

In [45]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 텍스트 분류기

In [199]:
data = pd.read_csv('idbot/ChatBotData.csv')
train_data=data[["Q","Rule?"]]
train_data.tail()

,Q,Rule?
9029,이번 주 행사 뭐 있어?,14
9030,행사가… 뭐가 있더라?,14
9031,이번주 무슨 행사 있다고?,14
9032,행사가?,14
9033,행사… 행사 좀…,14


In [200]:
questions = []
for sentence in train_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

In [201]:
questions[-1]

'행사… 행사 좀…'

In [202]:
answers = []
for answer in train_data["Rule?"]:
  answers.append(answer)

In [203]:
answers[-1]

14

In [204]:
# 최대 길이를 40으로 정의
MAX_LENGTH = 40

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_for_class(inputs,outputs):
  tokenized_inputs=[]

  for sentence in inputs:
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence = START_TOKEN + tokenizer.encode(sentence) + END_TOKEN

    tokenized_inputs.append(sentence)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, outputs

In [205]:
questions, answers = tokenize_for_class(questions,answers)

In [206]:
# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices((questions,answers))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [207]:
questions[0]

array([8133, 3926, 2807, 5132, 7909, 7910, 8134,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [208]:
modelA = tf.keras.Sequential([
        tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        input_length = 40,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax')
])

In [209]:
modelA.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=["accuracy"])

In [210]:
modelA.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_21 (Embedding)    (None, 40, 64)            520640    
                                                                 
 bidirectional_26 (Bidirecti  (None, 128)              66048     
 onal)                                                           
                                                                 
 dropout_28 (Dropout)        (None, 128)               0         
                                                                 
 dense_77 (Dense)            (None, 64)                8256      
                                                                 
 dense_78 (Dense)            (None, 15)                975       
                                                                 
Total params: 595,919
Trainable params: 595,919
Non-trainable params: 0
_______________________________________________

In [211]:
modelB = tf.keras.Sequential([
        tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        input_length = 40,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax')
])

In [212]:
modelB.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=["accuracy"])

In [213]:
modelB.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_22 (Embedding)    (None, 40, 64)            520640    
                                                                 
 bidirectional_27 (Bidirecti  (None, 512)              657408    
 onal)                                                           
                                                                 
 dropout_29 (Dropout)        (None, 512)               0         
                                                                 
 dense_79 (Dense)            (None, 256)               131328    
                                                                 
 dense_80 (Dense)            (None, 15)                3855      
                                                                 
Total params: 1,313,231
Trainable params: 1,313,231
Non-trainable params: 0
___________________________________________

In [214]:
modelC = tf.keras.Sequential([
        tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=64,
        input_length = 40,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(15, activation='softmax')
])

In [215]:
modelC.compile(loss='sparse_categorical_crossentropy',optimizer='rmsprop',metrics=["accuracy"])

In [216]:
modelC.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_23 (Embedding)    (None, 40, 64)            520640    
                                                                 
 bidirectional_28 (Bidirecti  (None, 40, 128)          66048     
 onal)                                                           
                                                                 
 bidirectional_29 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dropout_30 (Dropout)        (None, 128)               0         
                                                                 
 dense_81 (Dense)            (None, 64)                8256      
                                                                 
 dense_82 (Dense)            (None, 64)              

In [217]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='loss',  # 모니터 기준 설정 (val loss) 
                              patience=10,         # 10회 Epoch동안 개선되지 않는다면 종료
                             )

In [218]:
modelA.fit(dataset, epochs=20,callbacks=[earlystopping])

Epoch 1/20
142/142 [==============================] - 9s 12ms/step - loss: 0.2965 - accuracy: 0.9750
Epoch 2/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0854 - accuracy: 0.9816
Epoch 3/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0637 - accuracy: 0.9822
Epoch 4/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0580 - accuracy: 0.9826
Epoch 5/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0518 - accuracy: 0.9827
Epoch 6/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0466 - accuracy: 0.9836
Epoch 7/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0451 - accuracy: 0.9848
Epoch 8/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0428 - accuracy: 0.9862
Epoch 9/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0379 - accuracy: 0.9868
Epoch 10/20
142/142 [==============================] - 2s 12ms/step - loss: 0.0354 - accura

In [219]:
modelB.fit(dataset, epochs=20,callbacks=[earlystopping])

Epoch 1/20
142/142 [==============================] - 8s 13ms/step - loss: 0.2119 - accuracy: 0.9745
Epoch 2/20
142/142 [==============================] - 2s 13ms/step - loss: 0.0771 - accuracy: 0.9817
Epoch 3/20
142/142 [==============================] - 2s 13ms/step - loss: 0.0593 - accuracy: 0.9830
Epoch 4/20
142/142 [==============================] - 2s 13ms/step - loss: 0.0505 - accuracy: 0.9847
Epoch 5/20
142/142 [==============================] - 2s 13ms/step - loss: 0.0436 - accuracy: 0.9863
Epoch 6/20
142/142 [==============================] - 2s 14ms/step - loss: 0.0390 - accuracy: 0.9869
Epoch 7/20
142/142 [==============================] - 2s 14ms/step - loss: 0.0310 - accuracy: 0.9892
Epoch 8/20
142/142 [==============================] - 2s 13ms/step - loss: 0.0242 - accuracy: 0.9920
Epoch 9/20
142/142 [==============================] - 2s 13ms/step - loss: 0.0215 - accuracy: 0.9930
Epoch 10/20
142/142 [==============================] - 2s 13ms/step - loss: 0.0188 - accura

In [220]:
modelC.fit(dataset, epochs=20,callbacks=[earlystopping])

Epoch 1/20
142/142 [==============================] - 17s 23ms/step - loss: 0.2439 - accuracy: 0.9744
Epoch 2/20
142/142 [==============================] - 3s 22ms/step - loss: 0.0824 - accuracy: 0.9816
Epoch 3/20
142/142 [==============================] - 3s 22ms/step - loss: 0.0630 - accuracy: 0.9822
Epoch 4/20
142/142 [==============================] - 3s 22ms/step - loss: 0.0550 - accuracy: 0.9825
Epoch 5/20
142/142 [==============================] - 3s 23ms/step - loss: 0.0513 - accuracy: 0.9832
Epoch 6/20
142/142 [==============================] - 4s 28ms/step - loss: 0.0480 - accuracy: 0.9841
Epoch 7/20
142/142 [==============================] - 3s 23ms/step - loss: 0.0446 - accuracy: 0.9834
Epoch 8/20
142/142 [==============================] - 3s 22ms/step - loss: 0.0409 - accuracy: 0.9841
Epoch 9/20
142/142 [==============================] - 3s 22ms/step - loss: 0.0397 - accuracy: 0.9856
Epoch 10/20
142/142 [==============================] - 3s 22ms/step - loss: 0.0371 - accur

In [221]:
data = pd.read_csv('TestData.csv',encoding='cp949')
test_data=data[["Q","Rule?"]]
test_data.tail()

,Q,Rule?
55,진수 변환기 할래,13
56,6진수는 가능해?,13
57,이번 주 학사일정 뭐가 있지?,14
58,이번 달 행사가 좀 있지 않나?,14
59,그거 언제 행사더라?,14


In [222]:
qs = []
for sentence in test_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    qs.append(sentence)

In [223]:
ans = []
for answer in test_data["Rule?"]:
  ans.append(answer)

In [224]:
qs, ans = tokenize_for_class(qs,ans)

In [225]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

test = tf.data.Dataset.from_tensor_slices((qs,ans))

test = test.cache()
test = test.batch(BATCH_SIZE)
test = test.prefetch(tf.data.experimental.AUTOTUNE)

In [226]:
modelA.evaluate(test)

1/1 [==============================] - 2s 2s/step - loss: 9.0377 - accuracy: 0.5667


[9.037744522094727, 0.5666666626930237]

In [227]:
modelB.evaluate(test)

1/1 [==============================] - 2s 2s/step - loss: 7.4410 - accuracy: 0.6333


[7.441048622131348, 0.6333333253860474]

In [228]:
modelC.evaluate(test)

1/1 [==============================] - 6s 6s/step - loss: 6.3901 - accuracy: 0.5167


[6.3900861740112305, 0.5166666507720947]

In [229]:
def classify(sentence,model_classify):
  sentence = [START_TOKEN + tokenizer.encode(sentence) + END_TOKEN]
  sentence = tf.keras.preprocessing.sequence.pad_sequences(
      sentence, maxlen=MAX_LENGTH, padding='post')
  vec = model_classify.predict(sentence)
  c = np.argmax(vec)
  return c

In [230]:
classify("",modelC)

0

In [232]:
modelB.save("classifier.h5")

# 실사용용 챗봇을 구현해봐요

In [233]:
def chat(sentence):
  k = classify(sentence,modelB)
  if k==0:
    return predict(sentence)
  return k

In [234]:
print("이디봇: 저는 이디봇이고 너는 얘기를 해")
while(True):
  
  s = input("나: ")
  if s=="!종료":
    break
  print("이디봇: "+str(chat(s)))

이디봇: 저는 이디봇이고 너는 얘기를 해
나: 집 보내줘


이디봇: 지금 그러고 있어요 .
나: 특별실 여행할거다
이디봇: 메인 화면에서 특별실 신청 버튼을 누른 뒤 ,  사용 목적과 사용 장소 사용자를 올바르게 적은 뒤 선생님을 선택하면 돼요 .
나: 특별실 넣어줘
이디봇: 메인 화면에서 특별실 신청 버튼을 누른 뒤 ,  사용 목적과 사용 장소 사용자를 올바르게 적은 뒤 선생님을 선택하면 돼요 .
나: 특별실 신청할래
이디봇: 5
나: 특별실 넣어줄래?
이디봇: 5
나: 물리학 과목 교사는?
이디봇: 11
나: 물리학 과목 선생님은?
이디봇: 2
나: 정보과학 과목 교사는?
이디봇: 2
나: 정보과학 과목 선생님은?
이디봇: 2
나: 그렇군 음음.
이디봇: 문득 생각나고 잊혀지지 않나봐요 .
나: !종료
